In [54]:
import nltk as nltk
from nltk import ngrams
import string
import pymorphy2

Сначала научимся делать n-граммные теггеры для английского языка и на имеющемся в NLTK корпусе.

Подготовим тренировочную выборку и тестовую выборку:

In [14]:
from nltk.corpus import brown
nltk.download('brown')
news_train_tagged = brown.tagged_sents(categories='news')
news_train_raw = brown.sents(categories='news')
gov_test_tagged = brown.tagged_sents(categories='government')
gov_test_raw = brown.sents(categories='government')

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!


Обучим униграммный теггер и запустим его на предложении из того же корпуса, на котором он учился:

In [15]:
unigram_tagger = nltk.UnigramTagger(news_train_tagged)
unigram_tagger.tag(news_train_raw[2007])

[('Various', 'JJ'),
 ('of', 'IN'),
 ('the', 'AT'),
 ('apartments', 'NNS'),
 ('are', 'BER'),
 ('of', 'IN'),
 ('the', 'AT'),
 ('terrace', 'NN'),
 ('type', 'NN'),
 (',', ','),
 ('being', 'BEG'),
 ('on', 'IN'),
 ('the', 'AT'),
 ('ground', 'NN'),
 ('floor', 'NN'),
 ('so', 'QL'),
 ('that', 'CS'),
 ('entrance', 'NN'),
 ('is', 'BEZ'),
 ('direct', 'JJ'),
 ('.', '.')]

Теперь запустим его на незнакомом предложении:

In [16]:
unigram_tagger.tag(gov_test_raw[2])

[('Such', 'JJ'),
 ('measures', 'NNS'),
 ('are', 'BER'),
 ('essential', 'JJ'),
 ('to', 'TO'),
 ('its', 'PP$'),
 ('job', 'NN'),
 ('of', 'IN'),
 ('presenting', None),
 ('business', 'NN'),
 ('and', 'CC'),
 ('Government', 'NN-TL'),
 ('with', 'IN'),
 ('the', 'AT'),
 ('facts', 'NNS'),
 ('required', 'VBN'),
 ('to', 'TO'),
 ('meet', 'VB'),
 ('the', 'AT'),
 ('objective', 'NN'),
 ('of', 'IN'),
 ('expanding', 'VBG'),
 ('business', 'NN'),
 ('and', 'CC'),
 ('improving', 'VBG'),
 ('the', 'AT'),
 ('operation', 'NN'),
 ('of', 'IN'),
 ('the', 'AT'),
 ('economy', 'NN'),
 ('.', '.')]

Проверим эффективность обучения униграммного теггера:

In [17]:
unigram_tagger.evaluate(gov_test_tagged)

0.8082490694125533

Обучим биграммный теггер и запустим его на предложении из того же корпуса, на котором он учился:

In [18]:
bigram_tagger = nltk.BigramTagger(news_train_tagged)
bigram_tagger.tag(news_train_raw[2007])

[('Various', 'JJ'),
 ('of', 'IN'),
 ('the', 'AT'),
 ('apartments', 'NNS'),
 ('are', 'BER'),
 ('of', 'IN'),
 ('the', 'AT'),
 ('terrace', 'NN'),
 ('type', 'NN'),
 (',', ','),
 ('being', 'BEG'),
 ('on', 'IN'),
 ('the', 'AT'),
 ('ground', 'NN'),
 ('floor', 'NN'),
 ('so', 'CS'),
 ('that', 'CS'),
 ('entrance', 'NN'),
 ('is', 'BEZ'),
 ('direct', 'JJ'),
 ('.', '.')]

Теперь запустим его на незнакомом предложении:

In [19]:
bigram_tagger.tag(gov_test_raw[2])

[('Such', 'JJ'),
 ('measures', 'NNS'),
 ('are', 'BER'),
 ('essential', None),
 ('to', None),
 ('its', None),
 ('job', None),
 ('of', None),
 ('presenting', None),
 ('business', None),
 ('and', None),
 ('Government', None),
 ('with', None),
 ('the', None),
 ('facts', None),
 ('required', None),
 ('to', None),
 ('meet', None),
 ('the', None),
 ('objective', None),
 ('of', None),
 ('expanding', None),
 ('business', None),
 ('and', None),
 ('improving', None),
 ('the', None),
 ('operation', None),
 ('of', None),
 ('the', None),
 ('economy', None),
 ('.', None)]

Видим, что биграммный теггер распознаёт незнакомые слова гораздо хуже, чем униграммный. Чтобы справиться с этой проблемой, можно использовать комбинированный теггер.

Проверим эффективность обучения биграммного теггера:

In [20]:
bigram_tagger.evaluate(gov_test_tagged)

0.09094798693612105

В учебнике NLTK указано, что такой маленький процент определяемых биграммным теггером слов - норма: "Notice that the bigram tagger manages to tag every word in a sentence it saw during training, but does badly on an unseen sentence. As soon as it encounters a new word, it is unable to assign a tag. It cannot tag the following word (i.e., million) even if it was seen during training, simply because it never saw it during training with a None tag on the previous word. Consequently, the tagger fails to tag the rest of the sentence. Its overall accuracy score is very low". Теперь понятно, почему у него такой низкий процент определяемости: как только ему встречается незнакомое слово, он приписывает ему тег None. И следующее слово определяется уже при условии того, что у предыдущего токена был тег None - а такое теггер в тренировочной выборке не встречал; соответственно, новому слову он тоже поставит тег None. Ошибка этой системы похожа на снежный ком - невозможность определить весь текст начинает накапливаться из-за одной ошибки. С подобными ошибками можно отчасти справиться, комбинируя несколько теггеров сразу.

Обучим триграммный теггер и запустим его на предложении из того же корпуса, на котором он учился:

In [21]:
trigram_tagger = nltk.TrigramTagger(news_train_tagged)
trigram_tagger.tag(news_train_raw[2007])

[('Various', 'JJ'),
 ('of', 'IN'),
 ('the', 'AT'),
 ('apartments', 'NNS'),
 ('are', 'BER'),
 ('of', 'IN'),
 ('the', 'AT'),
 ('terrace', 'NN'),
 ('type', 'NN'),
 (',', ','),
 ('being', 'BEG'),
 ('on', 'IN'),
 ('the', 'AT'),
 ('ground', 'NN'),
 ('floor', 'NN'),
 ('so', 'CS'),
 ('that', 'CS'),
 ('entrance', 'NN'),
 ('is', 'BEZ'),
 ('direct', 'JJ'),
 ('.', '.')]

Теперь запустим его на незнакомом предложении:

In [22]:
trigram_tagger.tag(gov_test_raw[2])

[('Such', 'JJ'),
 ('measures', None),
 ('are', None),
 ('essential', None),
 ('to', None),
 ('its', None),
 ('job', None),
 ('of', None),
 ('presenting', None),
 ('business', None),
 ('and', None),
 ('Government', None),
 ('with', None),
 ('the', None),
 ('facts', None),
 ('required', None),
 ('to', None),
 ('meet', None),
 ('the', None),
 ('objective', None),
 ('of', None),
 ('expanding', None),
 ('business', None),
 ('and', None),
 ('improving', None),
 ('the', None),
 ('operation', None),
 ('of', None),
 ('the', None),
 ('economy', None),
 ('.', None)]

Видим, что триграммный теггер распознаёт незнакомые слова ещё хуже, чем биграммный, и гораздо хуже, чем униграммный.

Проверим эффективность обучения триграммного теггера:

In [23]:
trigram_tagger.evaluate(gov_test_tagged)

0.052811728967297515

Теперь объединим все теггеры в одну систему, которая смогла бы обращаться к следующему теггеру, если какой-то один не сработал или сработал плохо.

In [24]:
t0 = nltk.DefaultTagger('NN')
t1 = nltk.UnigramTagger(gov_test_tagged, backoff=t0)
t2 = nltk.BigramTagger(gov_test_tagged, backoff=t1)
t3 = nltk.TrigramTagger(gov_test_tagged, backoff=t2)
t3.evaluate(gov_test_tagged)

0.9796340402470157

Видим, что процент обучаемости значительно повысился!

In [25]:
t3.tag(gov_test_raw[2])

[('Such', 'JJ'),
 ('measures', 'NNS'),
 ('are', 'BER'),
 ('essential', 'JJ'),
 ('to', 'TO'),
 ('its', 'PP$'),
 ('job', 'NN'),
 ('of', 'IN'),
 ('presenting', 'VBG'),
 ('business', 'NN'),
 ('and', 'CC'),
 ('Government', 'NN-TL'),
 ('with', 'IN'),
 ('the', 'AT'),
 ('facts', 'NNS'),
 ('required', 'VBN'),
 ('to', 'TO'),
 ('meet', 'VB'),
 ('the', 'AT'),
 ('objective', 'NN'),
 ('of', 'IN'),
 ('expanding', 'VBG'),
 ('business', 'NN'),
 ('and', 'CC'),
 ('improving', 'VBG'),
 ('the', 'AT'),
 ('operation', 'NN'),
 ('of', 'IN'),
 ('the', 'AT'),
 ('economy', 'NN'),
 ('.', '.')]

Результат работы объединённого теггера - полностью разобранное предложение!

## Обучение и разметка русскоязычного корпуса текстов

Данные: для обучения используем вручную или автоматически размеченные и затем вручную проверенные корпусы:

1. SynTagRus, скачанный в формате .conllu из [репозитория проекта Universal Dependencies](https://github.com/UniversalDependencies/UD_Russian-SynTagRus) при помощи команды wget. В общий файл 'ru-train.csv' вошли данные из файлов 'dev', 'train-a', 'train-b', 'train-c'. Тестовые данные вошли в 'ru-test.csv'.

2. Taiga, скаченный в формате .conllu из [репозитория проекта Universal Dependencies](https://github.com/UniversalDependencies/UD_Russian-Taiga).


Для удобства данные были преобразованы в два файла в формате CSV: первый содержит обучающую выборку, второй — тестовую выборку, на которой мы будем проверять нашу модель. 

Из обоих текстов были предварительно удалены знаки препинания с помощью `string.punctuation`: !"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~

In [26]:
import pandas as pd


russian_train_corpus = pd.read_csv('ru-train.csv', sep='\t')
russian_test_corpus = pd.read_csv('ru-test.csv', sep='\t')

russian_train_corpus.head(n=20)

,Словоформа,Часть речи
0,анкета,NOUN
1,начальник,NOUN
2,областного,ADJ
3,управления,NOUN
4,связи,NOUN
5,семен,PROPN
6,еремеевич,PROPN
7,был,AUX
8,человек,NOUN
9,простой,ADJ


In [27]:
russian_test_corpus.head(n=20)

,Wordform,POS
0,в,ADP
1,советский,ADJ
2,период,NOUN
3,времени,NOUN
4,число,NOUN
5,ит,PROPN
6,специалистов,NOUN
7,армении,PROPN
8,составляло,VERB
9,около,ADP


Преобразуем имеюшиеся данные в тот формат, который воспринимает теггер. Отдельно выводим количество уникальных единиц в выборках.

In [28]:
print([russian_train_corpus.values])
print('-----')
print(f'Количество уникальных словоформ (обучающая выборка): {len(russian_train_corpus)}')
print(f'Количество уникальных словоформ (тестовая выборка): {len(russian_test_corpus)}')

[array([['анкета', 'NOUN'],
       ['начальник', 'NOUN'],
       ['областного', 'ADJ'],
       ...,
       ['скорблю', 'VERB'],
       ['озираю', 'VERB'],
       ['пирующих', 'VERB']], dtype=object)]
-----
Количество уникальных словоформ (обучающая выборка): 145425
Количество уникальных словоформ (тестовая выборка): 35672


Дополнительно создадим несколько предложений, на которых можно будет вручную проверять работу теггеров. Предложения взяты из газетного корпуса НКРЯ.

In [33]:
sentence_1 = 'Британские врачи провели ряд исследований на мышах и выяснили, что продолжительность сна напрямую влияет на здоровье легких.'
sentence_2 = 'Жители Хвалынска в Саратовской области решили купить достойные украшения для новогодней елки в центре города.'
sentence_3 = 'Ученые полагают, что делать выводы рано, и намерены продолжить работу.'

### **UnigramTagger**

Тренируем модель на обучающем корпусе.

In [29]:
rus_unigram_tagger = nltk.UnigramTagger([russian_train_corpus.values])

Проверим его эффективность на тестовой выборке при помощи функции `accuracy_score()`. Функция возвращает значение точности. Сраниванием "золотой стандарт" ручной разметки тестовой выборки с тегами, присвоенными `UnigramTagger`.

In [30]:
from sklearn.metrics import accuracy_score

russian_test_corpus['Unigram tagger'] = russian_test_corpus['Wordform'].apply(lambda x: rus_unigram_tagger.tag([x])[0][1])
russian_test_corpus['Unigram tagger'].replace([None], 'None', inplace=True)
russian_test_corpus

,Wordform,POS,Unigram tagger
0,в,ADP,ADP
1,советский,ADJ,ADJ
2,период,NOUN,NOUN
3,времени,NOUN,NOUN
4,число,NOUN,NOUN
...,...,...,...
35667,га,NOUN,NOUN
35668,поголовье,NOUN,NOUN
35669,прирастает,VERB,VERB
35670,партийного,ADJ,ADJ


In [31]:
accuracy_score(russian_test_corpus['POS'], russian_test_corpus['Unigram tagger'])

0.7176216640502355

Можно увидеть, что теггер достаточно неплохо справляется с разметкой. Проверим его вручную на нескольких тестовых предложениях. Сначала все слова токенизируются и приводятся к нижнему регистру; из предложений удаляются знаки препинания.

In [34]:
rus_unigram_tagger.tag(sentence_1.lower().strip(string.punctuation).split())

[('британские', 'ADJ'),
 ('врачи', 'NOUN'),
 ('провели', 'VERB'),
 ('ряд', 'NOUN'),
 ('исследований', 'NOUN'),
 ('на', 'ADP'),
 ('мышах', 'NOUN'),
 ('и', 'CCONJ'),
 ('выяснили,', None),
 ('что', 'PRON'),
 ('продолжительность', 'NOUN'),
 ('сна', 'NOUN'),
 ('напрямую', 'ADV'),
 ('влияет', 'VERB'),
 ('на', 'ADP'),
 ('здоровье', 'NOUN'),
 ('легких', 'ADJ')]

In [35]:
rus_unigram_tagger.tag(sentence_2.lower().strip(string.punctuation).split())

[('жители', 'NOUN'),
 ('хвалынска', None),
 ('в', 'ADP'),
 ('саратовской', 'ADJ'),
 ('области', 'NOUN'),
 ('решили', 'VERB'),
 ('купить', 'VERB'),
 ('достойные', 'ADJ'),
 ('украшения', 'NOUN'),
 ('для', 'ADP'),
 ('новогодней', 'ADJ'),
 ('елки', 'NOUN'),
 ('в', 'ADP'),
 ('центре', 'NOUN'),
 ('города', 'NOUN')]

In [36]:
rus_unigram_tagger.tag(sentence_3.lower().strip(string.punctuation).split())

[('ученые', 'NOUN'),
 ('полагают,', None),
 ('что', 'PRON'),
 ('делать', 'VERB'),
 ('выводы', 'NOUN'),
 ('рано,', None),
 ('и', 'CCONJ'),
 ('намерены', 'ADJ'),
 ('продолжить', 'VERB'),
 ('работу', 'NOUN')]

Можно отметить, что теггер справляется с большинством слов. 

### **BigramTagger**

Теперь обучим биграммный теггер. Сначала будем использовать настройки по умолчанию.

In [37]:
rus_bigram_tagger_1 = nltk.BigramTagger([russian_train_corpus.values])

Посмотрим, насколько хорошо обучился биграммный теггер:

In [39]:
russian_test_corpus['Bigram tagger'] = russian_test_corpus['Wordform'].apply(lambda x: rus_bigram_tagger_1.tag([x])[0][1])
russian_test_corpus['Bigram tagger'].replace([None], 'None', inplace=True)
russian_test_corpus

,Wordform,POS,Unigram tagger,Bigram tagger
0,в,ADP,ADP,None
1,советский,ADJ,ADJ,None
2,период,NOUN,NOUN,None
3,времени,NOUN,NOUN,None
4,число,NOUN,NOUN,None
...,...,...,...,...
35667,га,NOUN,NOUN,None
35668,поголовье,NOUN,NOUN,None
35669,прирастает,VERB,VERB,None
35670,партийного,ADJ,ADJ,None


In [40]:
accuracy_score(russian_test_corpus['POS'], russian_test_corpus['Bigram tagger'])

2.8033191298497423e-05

Смотрим, были ли присвоены теги хотя бы каким-то словоформам:

In [41]:
russian_test_corpus['Bigram tagger'].value_counts(normalize=True) * 100

None    99.997197
NOUN     0.002803
Name: Bigram tagger, dtype: float64

Видим, что значение точности намного ниже. Можно предположить, что подобного рода теггеры хорошо работают на более крупных корпусах, где объем выборки >1 млн уникальных словоформ. Чтобы как-то улучшить ситуацию, можно попробовать использовать лемматизацию.

Указываем при обучении параметр `backoff`: в таком случае, когда биграммный теггер не сможет определить часть речи, мы подключим униграммный теггер для решения данной задачи.

In [42]:
rus_bigram_tagger_2 = nltk.BigramTagger([russian_train_corpus.values], backoff=rus_unigram_tagger)

Оценим его:

In [43]:
russian_test_corpus['Bigram + unigram tagger'] = russian_test_corpus['Wordform'].apply(lambda x: rus_bigram_tagger_2.tag([x])[0][1])
russian_test_corpus['Bigram + unigram tagger'].replace([None], 'None', inplace=True)
russian_test_corpus

,Wordform,POS,Unigram tagger,Bigram tagger,Bigram + unigram tagger
0,в,ADP,ADP,None,ADP
1,советский,ADJ,ADJ,None,ADJ
2,период,NOUN,NOUN,None,NOUN
3,времени,NOUN,NOUN,None,NOUN
4,число,NOUN,NOUN,None,NOUN
...,...,...,...,...,...
35667,га,NOUN,NOUN,None,NOUN
35668,поголовье,NOUN,NOUN,None,NOUN
35669,прирастает,VERB,VERB,None,VERB
35670,партийного,ADJ,ADJ,None,ADJ


In [44]:
accuracy_score(russian_test_corpus['POS'], russian_test_corpus['Bigram + unigram tagger'])

0.7176216640502355

Видим, что значение точности увеличилось до того же значения, которое было вычислено для униграммной модели.

Протестируем модель на одном из тестовых предложений.

In [45]:
rus_bigram_tagger_2.tag(sentence_1.lower().strip(string.punctuation).split())

[('британские', 'ADJ'),
 ('врачи', 'NOUN'),
 ('провели', 'VERB'),
 ('ряд', 'NOUN'),
 ('исследований', 'NOUN'),
 ('на', 'ADP'),
 ('мышах', 'NOUN'),
 ('и', 'CCONJ'),
 ('выяснили,', None),
 ('что', 'PRON'),
 ('продолжительность', 'NOUN'),
 ('сна', 'NOUN'),
 ('напрямую', 'ADV'),
 ('влияет', 'VERB'),
 ('на', 'ADP'),
 ('здоровье', 'NOUN'),
 ('легких', 'ADJ')]

Теперь добавим лемматизацию при помощи `pymorphy2`.

In [55]:
morph = pymorphy2.MorphAnalyzer()
russian_train_corpus['Словоформа'] = russian_train_corpus['Словоформа'].apply(lambda x: morph.parse(x)[0].normal_form)
russian_test_corpus['Wordform'] = russian_test_corpus['Wordform'].apply(lambda x: morph.parse(x)[0].normal_form)

In [56]:
rus_bigram_tagger_2 = nltk.BigramTagger([russian_train_corpus.values], backoff=rus_unigram_tagger)

russian_test_corpus['Bigram + unigram tagger'] = russian_test_corpus['Wordform'].apply(lambda x: rus_bigram_tagger_2.tag([x])[0][1])
russian_test_corpus['Bigram + unigram tagger'].replace([None], 'None', inplace=True)
russian_test_corpus

,Wordform,POS,Unigram tagger,Bigram tagger,Bigram + unigram tagger
0,в,ADP,ADP,None,ADP
1,советский,ADJ,ADJ,None,ADJ
2,период,NOUN,NOUN,None,NOUN
3,время,NOUN,NOUN,None,NOUN
4,число,NOUN,NOUN,None,NOUN
...,...,...,...,...,...
35667,га,NOUN,NOUN,None,NOUN
35668,поголовье,NOUN,NOUN,None,NOUN
35669,прирастать,VERB,VERB,None,VERB
35670,партийный,ADJ,ADJ,None,ADJ


In [57]:
accuracy_score(russian_test_corpus['POS'], russian_test_corpus['Bigram + unigram tagger'])

0.743916797488226

### **TrigramTagger**

Обучим триграммный теггер. Поскольку биграммный теггер плохо показал себя с настройками по умолчанию, мы сразу используем дополнительные параметры и предварительно лемматизируем текст.

In [58]:
rus_trigram_tagger = nltk.TrigramTagger([russian_train_corpus.values], backoff=rus_unigram_tagger)

russian_test_corpus['Trigram + unigram tagger'] = russian_test_corpus['Wordform'].apply(lambda x: rus_trigram_tagger.tag([x])[0][1])
russian_test_corpus['Trigram + unigram tagger'].replace([None], 'None', inplace=True)
russian_test_corpus

,Wordform,POS,Unigram tagger,Bigram tagger,Bigram + unigram tagger,Trigram + unigram tagger
0,в,ADP,ADP,None,ADP,ADP
1,советский,ADJ,ADJ,None,ADJ,ADJ
2,период,NOUN,NOUN,None,NOUN,NOUN
3,время,NOUN,NOUN,None,NOUN,NOUN
4,число,NOUN,NOUN,None,NOUN,NOUN
...,...,...,...,...,...,...
35667,га,NOUN,NOUN,None,NOUN,NOUN
35668,поголовье,NOUN,NOUN,None,NOUN,NOUN
35669,прирастать,VERB,VERB,None,VERB,VERB
35670,партийный,ADJ,ADJ,None,ADJ,ADJ


In [59]:
accuracy_score(russian_test_corpus['POS'], russian_test_corpus['Trigram + unigram tagger'])

0.743916797488226

Вывод: биграммные и триграммные теггеры работают хуже для русскоязычных текстов по сравнению с униграммным теггеров. Униграммный теггер справляется с >70% словоформ, что можно считать хорошим резальутатом. Триграммный теггер в комбинации с предварительной лемматизацией и униграммным теггером способен дать более высокие показатели, которые чуть выше, чем показатели при таких же параметрах у биграммной модели.